In [1]:
import pandas as pd

In [3]:
files = [
    "data/swin_base_patch4_window7_224_imagenet_fp32_WindowAttention_1.00e-03.csv", 
    "data/swin_base_patch4_window7_224_imagenet_fp32_SwinTransformerBlock_1.00e-03.csv",
    "data/swin_base_patch4_window7_224_imagenet_fp32_Mlp_1.00e-03.csv",
    "data/vit_base_patch16_224_imagenet_fp32_Attention_1.00e-03.csv", 
    "data/vit_base_patch16_224_imagenet_fp32_Block_1.00e-03.csv",
    "data/vit_base_patch16_224_imagenet_fp32_Mlp_1.00e-03.csv",
]

In [ ]:
for file in files:
    print("="*50)
    if "SwinTransformerBlock" in file:
        print(f" [INFO] SwinTransformerBlock")
    elif "Mlp" in file:
        print(f" [INFO] Mlp")
    elif "WindowAttention" in file:
        print(f" [INFO] WindowAttention")
    else:
        print(f" [INFO] Unknown")
    
    df = pd.read_csv(file)

    accuracy_wo_fault = (df["prediction_without_fault"] == df["ground_truth"]).sum() / len(df)
    accuracy_with_fault = (df["prediction_with_fault"] == df["ground_truth"]).sum() / len(df)

    print(f" [INFO] Accuracy without fault: {accuracy_wo_fault * 100:.4f}%")
    print(f" [INFO] Accuracy with fault: {accuracy_with_fault * 100:.4f}%")
    print()
    count = 0
    for i, row in df.iterrows():
        out_wo_fault, out_with_fault = int(row["prediction_without_fault"]), int(row["prediction_with_fault"])
        if out_with_fault != out_wo_fault:
            count += 1
            print(f" [INFO] Critical error for image {i}. Expected {out_wo_fault} but got {out_with_fault}. Ground truth: {int(row['ground_truth'])}")

    print(f" [INFO] Total critical errors: {count}")

 [INFO] WindowAttention
 [INFO] Accuracy without fault: 85.27%
 [INFO] Accuracy with fault: 85.28%

 [INFO] Critical error for image 3066. Expected 45 but got 29. Ground truth: 61
 [INFO] Critical error for image 3417. Expected 68 but got 66. Ground truth: 68
 [INFO] Critical error for image 8671. Expected 173 but got 253. Ground truth: 173
 [INFO] Critical error for image 9335. Expected 184 but got 265. Ground truth: 186
 [INFO] Critical error for image 11116. Expected 222 but got 258. Ground truth: 222
 [INFO] Critical error for image 19670. Expected 108 but got 393. Ground truth: 393
 [INFO] Critical error for image 21793. Expected 876 but got 502. Ground truth: 435
 [INFO] Critical error for image 26728. Expected 729 but got 651. Ground truth: 534
 [INFO] Critical error for image 26733. Expected 567 but got 896. Ground truth: 534
 [INFO] Critical error for image 30140. Expected 702 but got 602. Ground truth: 602
 [INFO] Critical error for image 33937. Expected 678 but got 837. Grou

In [ ]:
dataset = "imagenet"
precision = "fp32"
models = [
    "swin_base_patch4_window7_224",
    "vit_base_patch16_224",
]

for model in models:
    file = f"data/{model}_{dataset}_{precision}_correct_predictions.csv"
    if model == "swin_base_patch4_window7_224":
        to_open = "data/swin_base_patch4_window7_224_imagenet_fp32_WindowAttention_1.00e-03.csv"
    elif model == "vit_base_patch16_224":
        top_open = "data/vit_base_patch16_224_imagenet_fp32_Attention_1.00e-03.csv"

    df = pd.read_csv(to_open)
    df = df[df["prediction_without_fault"] == df["ground_truth"]]
    df.to_csv(file, index=True)

In [5]:
import torch
import os

INDICES_IDX, VALUES_IDX = 0, 1

dataset = "imagenet"
precision = "fp32"
models = [
    "swin_base_patch4_window7_224",
    "vit_base_patch16_224",
]

final_dfs = []
for model in models:
    dfs = []
    logits = [f for f in os.listdir(f"data/top5prob") if f"{model}_{dataset}_{precision}" in f]
    
    # logit = next(iter(logits))
    # logit = torch.load(f"data/top5prob/{logit}")
    # print(logit.values[0])
    for logit_path in logits:
        logit = torch.load(f"data/top5prob/{logit_path}")
        dfs.append({
            "model": model,
            "dataset": dataset,
            "precision": precision,
            "top1_prob": logit[VALUES_IDX][0].item(),
            "top2_prob": logit[VALUES_IDX][1].item(),
            "top_diff": logit[VALUES_IDX][0].item() - logit[VALUES_IDX][1].item(),
            "top1_class": int(logit[INDICES_IDX][0].item()),
            "top2_class": int(logit[INDICES_IDX][1].item()),
        })

    df = pd.DataFrame(dfs)
    final_dfs.append((model, df))
    df.to_csv(f"data/{model}_{dataset}_{precision}_top5_prob.csv", index=False)



/tmp/ipykernel_2351118/3241280544.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  logit = torch.load(f"data/top5prob/{logit_path}")


In [26]:
for model, df in final_dfs:
    print("="*50)
    print(f"[INFO] {model}")
    # print(f" Min diff: {df['top_diff'].min():.4f}")
    # print(f" Max diff: {df['top_diff'].max():.4f}")
    # print(f" Mean diff: {df['top_diff'].mean():.4f}")
    # print(f" Median diff: {df['top_diff'].median():.4f}")
    # print(f" Q25% diff: {df['top_diff'].quantile(0.25):.4f}")
    # print(f" Q75% diff: {df['top_diff'].quantile(0.75):.4f}")
    # print(f" Q5% diff: {df['top_diff'].quantile(0.05):.4f}")
    # print(f" Q10% diff: {df['top_diff'].quantile(0.10):.4f}")
    # print(f" diff < 0.01: {(df['top_diff'] > 0.01).sum()}")
    print(f" top1_prob < 10%: {(df['top1_prob'] < 0.1).sum()}")
    print(f" top1_prob < 20%: {(df['top1_prob'] < 0.2).sum()}")
    print(f" top1_prob < 30%: {(df['top1_prob'] < 0.3).sum()}")
    print(f" top1_prob < 40%: {(df['top1_prob'] < 0.4).sum()}")
    print(f" top1_prob < 50%: {(df['top1_prob'] < 0.5).sum()}")


[INFO] swin_base_patch4_window7_224
 top1_prob < 10%: 6
 top1_prob < 20%: 128
 top1_prob < 30%: 443
 top1_prob < 40%: 1257
 top1_prob < 50%: 2671
[INFO] vit_base_patch16_224
 top1_prob < 10%: 7
 top1_prob < 20%: 88
 top1_prob < 30%: 416
 top1_prob < 40%: 1164
 top1_prob < 50%: 2671


In [35]:
for model, df in final_dfs:
    print("="*50)
    df_test = df.copy()
    df_test = df_test[(df_test["top1_prob"] < 0.2) & (df_test["top_diff"] <= 0.02)]
    for i, row in df_test.iterrows():
        print(f" [INFO] {model} - {row['top1_prob']:.4f} - {row['top2_prob']:.4f} - {row['top_diff']:.4f}")
    print(f" [INFO] Total: {len(df_test)}")

 [INFO] swin_base_patch4_window7_224 - 0.1061 - 0.0954 - 0.0107
 [INFO] swin_base_patch4_window7_224 - 0.0954 - 0.0816 - 0.0138
 [INFO] swin_base_patch4_window7_224 - 0.1340 - 0.1225 - 0.0115
 [INFO] swin_base_patch4_window7_224 - 0.1827 - 0.1822 - 0.0005
 [INFO] swin_base_patch4_window7_224 - 0.1675 - 0.1513 - 0.0162
 [INFO] swin_base_patch4_window7_224 - 0.1272 - 0.1131 - 0.0141
 [INFO] swin_base_patch4_window7_224 - 0.1779 - 0.1666 - 0.0113
 [INFO] swin_base_patch4_window7_224 - 0.1823 - 0.1652 - 0.0171
 [INFO] swin_base_patch4_window7_224 - 0.0703 - 0.0642 - 0.0061
 [INFO] swin_base_patch4_window7_224 - 0.1678 - 0.1577 - 0.0102
 [INFO] swin_base_patch4_window7_224 - 0.1482 - 0.1389 - 0.0093
 [INFO] swin_base_patch4_window7_224 - 0.0939 - 0.0839 - 0.0101
 [INFO] swin_base_patch4_window7_224 - 0.1093 - 0.1057 - 0.0036
 [INFO] swin_base_patch4_window7_224 - 0.1703 - 0.1648 - 0.0056
 [INFO] swin_base_patch4_window7_224 - 0.1364 - 0.1237 - 0.0127
 [INFO] swin_base_patch4_window7_224 - 0

In [39]:
machines = [
    "titanv",
    "a20001",
]

files = [
    "swin_base_patch4_window7_224_imagenet_fp32_WindowAttention_1.00e-03.csv", 
    "swin_base_patch4_window7_224_imagenet_fp32_SwinTransformerBlock_1.00e-03.csv",
    "swin_base_patch4_window7_224_imagenet_fp32_Mlp_1.00e-03.csv",
    "vit_base_patch16_224_imagenet_fp32_Attention_1.00e-03.csv", 
    "vit_base_patch16_224_imagenet_fp32_Block_1.00e-03.csv",
    "vit_base_patch16_224_imagenet_fp32_Mlp_1.00e-03.csv",
]

df_fi = []

for machine in machines:
    full_machine = f"carol{machine}"
    print("="*50)
    print(f" [INFO] {machine}")
    for file in files:
        path = f"data/vit_fi_res/{full_machine}/{file}"
        df = pd.read_csv(path)

        split = file.split("_imagenet_")
        model = split[0]
        microop = split[1].split("_")[1]

        print("-"*20)
        print
        if "SwinTransformerBlock" in file:
            print(f" [INFO] SwinTransformerBlock")
        elif "Mlp" in file:
            print(f" [INFO] Mlp")
        elif "WindowAttention" in file:
            print(f" [INFO] WindowAttention")
        elif "Attention" in file:
            print(f" [INFO] Attention")
        elif "Block" in file:
            print(f" [INFO] Block")
        else:
            print(f" [INFO] Unknown")

        accuracy_wo_fault = (df["prediction_without_fault"] == df["ground_truth"]).sum() / len(df)
        accuracy_with_fault = (df["prediction_with_fault"] == df["ground_truth"]).sum() / len(df)

        print(f" [INFO] Accuracy without fault: {accuracy_wo_fault * 100:.4f}%")
        print(f" [INFO] Accuracy with fault: {accuracy_with_fault * 100:.4f}%")
        print()
        count = 0
        for i, row in df.iterrows():
            out_wo_fault, out_with_fault = int(row["prediction_without_fault"]), int(row["prediction_with_fault"])
            if out_with_fault != out_wo_fault:
                count += 1
                print(f" [INFO] Critical error for image {i}. Expected {out_wo_fault} but got {out_with_fault}. Ground truth: {int(row['ground_truth'])}")
                df_fi.append({
                    "model": model,
                    "microop": microop,
                    "machine": machine,
                    "image_id": i,
                    "prediction_without_fault": out_wo_fault,
                    "prediction_with_fault": out_with_fault,
                })

        print(f" [INFO] Total critical errors: {count}")

df_fi = pd.DataFrame(df_fi)
df_fi.to_csv("data/fi_critical_errors.csv", index=False)
        

    

 [INFO] titanv
--------------------
 [INFO] WindowAttention
 [INFO] Accuracy without fault: 100.0000%
 [INFO] Accuracy with fault: 99.9906%

 [INFO] Critical error for image 7729. Expected 173 but got 253. Ground truth: 173
 [INFO] Critical error for image 20727. Expected 475 but got 15. Ground truth: 475
 [INFO] Critical error for image 21088. Expected 485 but got 482. Ground truth: 485
 [INFO] Critical error for image 30515. Expected 710 but got 767. Ground truth: 710
 [INFO] Total critical errors: 4
--------------------
 [INFO] SwinTransformerBlock
 [INFO] Accuracy without fault: 100.0000%
 [INFO] Accuracy with fault: 99.9789%

 [INFO] Critical error for image 3249. Expected 74 but got 73. Ground truth: 74
 [INFO] Critical error for image 5529. Expected 124 but got 123. Ground truth: 124
 [INFO] Critical error for image 7729. Expected 173 but got 253. Ground truth: 173
 [INFO] Critical error for image 19373. Expected 443 but got 618. Ground truth: 443
 [INFO] Critical error for imag

In [ ]:
crit_faults = pd.read_csv("data/fi_critical_errors.csv")
df_carola20001 = crit_faults[crit_faults["machine"] == "a20001"].reset_index(drop=True)
df_caroltitanv = crit_faults[crit_faults["machine"] == "titanv"].reset_index(drop=True)
df_carola20001 = df_carola20001[["model", "microop", "image_id", "prediction_without_fault", "prediction_with_fault"]]
df_caroltitanv = df_caroltitanv[["model", "microop", "image_id", "prediction_without_fault", "prediction_with_fault"]]

if pd.DataFrame.equals(df_carola20001, df_caroltitanv):
    print("[INFO] Dataframes are equal")
    crit_faults = df_carola20001

# crit_faults = crit_faults.groupby(["model", 'image_id']).agg({
#     "microop": "count",
# })
crit_faults.to_csv("data/fi_critical_images.csv", index=False)
# crit_faults.to_excel("data/fi_critical_errors_grouped.xlsx")

[INFO] Dataframes are equal


,model,microop,image_id,prediction_without_fault,prediction_with_fault
0,swin_base_patch4_window7_224,WindowAttention,7729,173,253
1,swin_base_patch4_window7_224,WindowAttention,20727,475,15
2,swin_base_patch4_window7_224,WindowAttention,21088,485,482
3,swin_base_patch4_window7_224,WindowAttention,30515,710,767
4,swin_base_patch4_window7_224,SwinTransformerBlock,3249,74,73
5,swin_base_patch4_window7_224,SwinTransformerBlock,5529,124,123
6,swin_base_patch4_window7_224,SwinTransformerBlock,7729,173,253
7,swin_base_patch4_window7_224,SwinTransformerBlock,19373,443,618
8,swin_base_patch4_window7_224,SwinTransformerBlock,23333,541,542
9,swin_base_patch4_window7_224,SwinTransformerBlock,29164,678,837


In [68]:
test = crit_faults.groupby(["model", "microop"]).agg({
    "image_id": "count",
})
test

image_id
model                        microop                       
swin_base_patch4_window7_224 Mlp                          6
                             SwinTransformerBlock         9
                             WindowAttention              4
vit_base_patch16_224         Attention                   11
                             Block                        8
                             Mlp                          8

In [87]:
def concat(x):
    return ", ".join(x)

test_unique = crit_faults.groupby(["image_id"]).agg({
    "image_id": "count",
    "model": "unique",
    "microop": "unique",
})
test_unique = test_unique.rename(columns={"image_id": "count"}).reset_index()
test_unique.to_csv("data/fi_critical_images_unique.csv", index=False)
test_unique

,image_id,count,model,microop
0,2650,1,[vit_base_patch16_224],[Attention]
1,2917,1,[swin_base_patch4_window7_224],[Mlp]
2,2978,1,[vit_base_patch16_224],[Mlp]
3,3249,1,[swin_base_patch4_window7_224],[SwinTransformerBlock]
4,5529,1,[swin_base_patch4_window7_224],[SwinTransformerBlock]
5,6645,2,[vit_base_patch16_224],"[Attention, Block]"
6,6787,1,[vit_base_patch16_224],[Mlp]
7,7729,2,[swin_base_patch4_window7_224],"[WindowAttention, SwinTransformerBlock]"
8,9838,1,[swin_base_patch4_window7_224],[Mlp]
9,12912,3,[vit_base_patch16_224],"[Attention, Block, Mlp]"


In [89]:
test2 = pd.read_csv("data/fi_critical_images_unique.csv")
test2

,image_id,count,model,microop
0,2650,1,['vit_base_patch16_224'],['Attention']
1,2917,1,['swin_base_patch4_window7_224'],['Mlp']
2,2978,1,['vit_base_patch16_224'],['Mlp']
3,3249,1,['swin_base_patch4_window7_224'],['SwinTransformerBlock']
4,5529,1,['swin_base_patch4_window7_224'],['SwinTransformerBlock']
5,6645,2,['vit_base_patch16_224'],['Attention' 'Block']
6,6787,1,['vit_base_patch16_224'],['Mlp']
7,7729,2,['swin_base_patch4_window7_224'],['WindowAttention' 'SwinTransformerBlock']
8,9838,1,['swin_base_patch4_window7_224'],['Mlp']
9,12912,3,['vit_base_patch16_224'],['Attention' 'Block' 'Mlp']
